In [1]:
!pip install pathway bokeh --quiet

import pandas as pd
import numpy as np
import pathway as pw
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from datetime import datetime

output_notebook()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Load data
df = pd.read_csv("/content/dataset.csv")  # or the correct path if you're running locally

# Combine Date + Time
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format="%d-%m-%Y %H:%M:%S")

# Encode VehicleType (car=1.0, bike=0.5, truck=1.5)
vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_map)

# Encode TrafficConditionNearby (low=0, medium=1, high=2)
traffic_map = {'low': 0, 'medium': 1, 'high': 2}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

# Normalize required columns (for Model 2)
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['NormQueue'] = df['QueueLength'] / df['QueueLength'].max()
df['NormTraffic'] = df['TrafficLevel'] / 2.0  # since 2 is max

df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,VehicleWeight,TrafficLevel,OccupancyRate,NormQueue,NormTraffic
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,1.0,0.0,0.105719,0.066667,0.0
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00,1.0,0.0,0.110919,0.066667,0.0
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00,1.0,0.0,0.138648,0.133333,0.0
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00,1.0,0.0,0.185442,0.133333,0.0
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00,0.5,0.0,0.259965,0.133333,0.0


In [3]:
def baseline_price(prev_price, occupancy, capacity, alpha=2.0):
    occ_rate = occupancy / capacity
    return prev_price + alpha * occ_rate

# Apply to dataset (we simulate price per lot sequentially)
initial_price = 10.0
baseline_prices = []
lot_price_map = {}  # Track price per lot

for idx, row in df.iterrows():
    lot = row['SystemCodeNumber']
    if lot not in lot_price_map:
        lot_price_map[lot] = initial_price
    new_price = baseline_price(lot_price_map[lot], row['Occupancy'], row['Capacity'])
    baseline_prices.append(round(new_price, 2))
    lot_price_map[lot] = new_price

df['BaselinePrice'] = baseline_prices

In [4]:
# Demand Function:
def calculate_demand(row, α=0.5, β=1.0, γ=1.2, δ=1.5, ε=0.8):
    demand = (
        α * row['OccupancyRate'] +
        β * row['NormQueue'] -
        γ * row['NormTraffic'] +
        δ * row['IsSpecialDay'] +
        ε * row['VehicleWeight']
    )
    return demand

# Calculate raw and normalized demand
df['RawDemand'] = df.apply(calculate_demand, axis=1)
df['NormalizedDemand'] = (df['RawDemand'] - df['RawDemand'].min()) / (df['RawDemand'].max() - df['RawDemand'].min())

# Price Function
def dynamic_price(norm_demand, base_price=10, λ=1.0):
    price = base_price * (1 + λ * norm_demand)
    return round(np.clip(price, 5, 20), 2)

df['DynamicPrice'] = df['NormalizedDemand'].apply(lambda x: dynamic_price(x))

In [5]:
# Example static line plot of price vs time for one lot
import random
sample_lot = random.choice(df['SystemCodeNumber'].unique())
lot_df = df[df['SystemCodeNumber'] == sample_lot]

p = figure(x_axis_type='datetime', title=f"Price vs Time for {sample_lot}", width=800)
source = ColumnDataSource(lot_df)
p.line(x='Timestamp', y='BaselinePrice', source=source, color='blue', legend_label="Baseline Price")
p.line(x='Timestamp', y='DynamicPrice', source=source, color='green', legend_label="Demand Price")
p.legend.location = "top_left"
show(p)


In [6]:
from math import radians, cos, sin, asin, sqrt

# Haversine distance between two lat/lon points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of earth in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return R * 2 * asin(sqrt(a))

# Create lot location dictionary
lot_locations = df.groupby("SystemCodeNumber")[["Latitude", "Longitude"]].first().to_dict("index")

# Build proximity list
proximity_map = {}
for lot1 in lot_locations:
    proximity_map[lot1] = []
    for lot2 in lot_locations:
        if lot1 == lot2: continue
        dist = haversine(
            lot_locations[lot1]["Latitude"], lot_locations[lot1]["Longitude"],
            lot_locations[lot2]["Latitude"], lot_locations[lot2]["Longitude"]
        )
        if dist < 1.0:  # Only within 1 km
            proximity_map[lot1].append(lot2)


In [7]:
# Simulate final price considering nearby lot prices
final_prices = []

for idx, row in df.iterrows():
    lot = row['SystemCodeNumber']
    price = row['DynamicPrice']
    occupancy = row['Occupancy']
    capacity = row['Capacity']

    nearby_prices = df[(df['SystemCodeNumber'].isin(proximity_map[lot])) & (df['Timestamp'] == row['Timestamp'])]['DynamicPrice']

    if not nearby_prices.empty:
        nearby_avg = nearby_prices.mean()

        # Competitive logic
        if occupancy >= capacity * 0.95 and price > nearby_avg:
            price -= 1  # Reduce price if you're full & others are cheaper
        elif nearby_avg > price:
            price += 0.5  # Slightly increase if you're cheaper

    final_prices.append(round(np.clip(price, 5, 20), 2))

df['FinalPrice'] = final_prices


In [13]:
@pw.udf
def compute_price(cap, occ, q, traffic, special, weight):
    # Provide fallback defaults in case any value is None
    cap = cap or 1  # avoid division by 0
    occ = occ or 0
    q = q or 0
    traffic = traffic or 0
    special = special or 0
    weight = weight or 1.0

    occ_rate = occ / cap
    demand = (
        0.5 * occ_rate +
        1.0 * q -
        1.2 * traffic +
        1.5 * special +
        0.8 * weight
    )
    norm_demand = min(max((demand + 5) / 10, 0), 1)
    price = 10 * (1 + norm_demand)
    return round(np.clip(price, 5, 20), 2)


In [14]:
# Create input table
input_tbl = pw.debug.table_from_pandas(df_stream)

# Apply the new compute_price()
output_tbl = input_tbl.select(
    SystemCodeNumber=pw.this.SystemCodeNumber,
    Timestamp=pw.this.Timestamp,
    Price=compute_price(
        pw.this.Capacity,
        pw.this.Occupancy,
        pw.this.QueueLength,
        pw.this.TrafficLevel,
        pw.this.IsSpecialDay,
        pw.this.VehicleWeight
    )
)

# Show the output
pw.debug.compute_and_print(output_tbl, include_id=False)

SystemCodeNumber | Timestamp           | Price
BHMBCCMKT01      | 2016-10-04 07:59:00 | 16.85
BHMBCCMKT01      | 2016-10-04 08:25:00 | 16.86
BHMBCCMKT01      | 2016-10-04 08:59:00 | 17.87
BHMBCCMKT01      | 2016-10-04 09:32:00 | 17.89
BHMBCCMKT01      | 2016-10-04 09:59:00 | 17.53
BHMBCCMKT01      | 2016-10-04 10:26:00 | 18.95
BHMBCCMKT01      | 2016-10-04 10:59:00 | 19.99
BHMBCCMKT01      | 2016-10-04 11:25:00 | 20
BHMBCCMKT01      | 2016-10-04 11:59:00 | 20
BHMBCCMKT01      | 2016-10-04 12:29:00 | 20
BHMBCCMKT01      | 2016-10-04 13:02:00 | 20
BHMBCCMKT01      | 2016-10-04 13:29:00 | 20
BHMBCCMKT01      | 2016-10-04 14:02:00 | 18.61
BHMBCCMKT01      | 2016-10-04 14:29:00 | 19.39
BHMBCCMKT01      | 2016-10-04 14:57:00 | 18.57
BHMBCCMKT01      | 2016-10-04 15:30:00 | 17.94
BHMBCCMKT01      | 2016-10-04 16:04:00 | 16.54
BHMBCCMKT01      | 2016-10-04 16:31:00 | 17.92
BHMBCCMKT01      | 2016-10-05 07:57:00 | 16.85
BHMBCCMKT01      | 2016-10-05 08:30:00 | 16.85
BHMBCCMKT01      | 2016-10-0

In [15]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

# Filter output for one lot
lot_name = "Shopping"
lot_df = df[df['SystemCodeNumber'] == lot_name]

p = figure(x_axis_type='datetime', title=f"Price Over Time: {lot_name}", width=800)
source = ColumnDataSource(lot_df)

p.line(x='Timestamp', y='FinalPrice', source=source, color='green', legend_label="Price")
p.yaxis.axis_label = "Price ($)"
p.xaxis.axis_label = "Time"

show(p)

In [16]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

output_notebook()

# Choose a few lots to visualize
lots = df['SystemCodeNumber'].unique()[:4]  # pick 4 for display

plots = []

for lot in lots:
    lot_df = df[df['SystemCodeNumber'] == lot].copy()
    lot_df.sort_values('Timestamp', inplace=True)

    source = ColumnDataSource(lot_df)

    p = figure(x_axis_type='datetime', title=f"Dynamic Price: {lot}", width=400, height=300)
    p.line(x='Timestamp', y='FinalPrice', source=source, color='navy', legend_label='Final Price')
    p.line(x='Timestamp', y='BaselinePrice', source=source, color='gray', legend_label='Baseline', line_dash='dashed')

    p.legend.location = 'top_left'
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Price ($)'

    plots.append(p)

grid = gridplot([plots[:2], plots[2:]])  # 2x2 layout
show(grid)

In [ ]:
##Conclusion

## This notebook implements dynamic pricing for urban parking lots using real-time data and three levels of model complexity. Prices adapt to demand, traffic, and vehicle type using both static logic and real-time streaming powered by Pathway.
